In [1]:
from math import pi

import pandas as pd

from bokeh.io import show
from bokeh.models import BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter
from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data
from bokeh.palettes import viridis
from bokeh.io import output_notebook
output_notebook()
import picaso.justplotit as jpi

import picaso.picaso_data as ptest
import picaso.disort_disco as dtest

import numpy as np

Loading BokehJS ...

In [2]:
DE = True

# Toon quadrature
Toon_quad = ptest.picaso_albedos(output_dir='Toon_quad.csv', delta_eddington=True, disort_data=True)
# Toon eddington
# Toon_edd = ptest.picaso_albedos(output_dir= 'Toon_eddd.csv',Toon_coefficients ="eddington", delta_eddington=DE)
# SH 2-stream
SH2 = ptest.picaso_albedos(output_dir='SH2.csv',method="SH", delta_eddington=DE, rayleigh=False, disort_data=True)
# SH 4-stream
SH4 = ptest.picaso_albedos(output_dir='SH4.csv',method='SH', stream=4, delta_eddington=True, rayleigh=False, disort_data=True)

YOU ARE REQUESTING RESAMPLING!!


NameError: name 'xint' is not defined

In [ ]:
disort = dtest.disort_albedos(output_dir='disort.csv')

In [ ]:
# read in data from picaso function
data_old = pd.read_csv('DLUGACH_TEST.csv',index_col=0)
data_disort = pd.read_csv('disort.csv',index_col=0)
data_SH2 = pd.read_csv('SH2.csv',index_col=0)
data_SH4 = pd.read_csv('SH4.csv',index_col=0)
data_quad = pd.read_csv('Toon_quad.csv',index_col=0)
data_edd = pd.read_csv('Toon_eddd.csv',index_col=0)

#If you want to compare two different files for the heatmap just do this
data_compare = data_old
compare_quad = (data_compare-data_quad)/data_compare*100   
compare_edd = (data_compare-data_edd)/data_compare*100   
compare_SH2 = (data_compare-data_SH2)/data_compare*100 
compare_SH4 = (data_compare-data_SH4)/data_compare*100  
compare_SH = (data_SH2-data_SH4)/data_SH2*100    

quad_SH4 = (data_quad-data_SH4)/data_quad*100 
quad_SH2 = (data_quad-data_SH2)/data_quad*100 
edd_SH2 = (data_edd-data_SH2)/data_edd*100 

disort_dlugach = (data_old-data_disort)/data_old*100 

In [ ]:
show(jpi.create_heat_map(disort_dlugach, rayleigh=True))

In [ ]:
show(jpi.create_heat_map(quad_SH4, rayleigh=False))

In [ ]:
show(jpi.create_heat_map(quad_SH2, rayleigh=False))

In [ ]:
show(jpi.create_heat_map(compare_SH, rayleigh=False))

In [13]:
import pickle as pk
toon_dir = '/Users/crooney/Documents/codes/pyDISORT/test/picaso_data/'
cdisort_dir = '/Users/crooney/Documents/codes/pyDISORT/test/cdisort_data/'
SH2_dir = '/Users/crooney/Documents/codes/pyDISORT/test/picaso_data/SH2/'
SH4_dir = '/Users/crooney/Documents/codes/pyDISORT/test/picaso_data/SH4/'

for w0 in [.9]:
    filename = 'data_0.900_%.3f.pk' % w0
    Toon = pk.load(open(toon_dir + filename,'rb'))
#     Disort_4 = pk.load(open(disort4_dir + filename,'rb'), encoding = 'bytes')
#     Disort_DE = pk.load(open(disortDE_dir + filename,'rb'), encoding = 'bytes')
    Cdisort = pk.load(open(cdisort_dir + filename,'rb'), encoding = 'bytes')
    SH2 = pk.load(open(SH2_dir + filename,'rb'), encoding = 'bytes')
    SH4 = pk.load(open(SH4_dir + filename,'rb'), encoding = 'bytes')

In [37]:
n=0
Toon['xint_at_top'].shape

(5, 1, 51)

In [15]:
SH2['xint_at_top'][:,0,0]

array([0.02374264, 0.02573551, 0.03209829, 0.0461495 , 0.07312606])

In [16]:
SH4['xint_at_top'][:,0,0]

array([0.02234446, 0.02354699, 0.02837123, 0.04306426, 0.08047989])

In [17]:
Cdisort[b'xint_at_top'][:,0,0,0]

array([0.02424406, 0.02311912, 0.02784911, 0.04265516, 0.07581476])

In [20]:
Toon['flux'][0,0,::2,0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [21]:
SH2['flux'][0,0,::2,0]

array([1.00000000e+00, 9.99976126e-01, 9.99924692e-01, 9.99813890e-01,
       9.99575216e-01, 9.99061202e-01, 9.97954692e-01, 9.95574950e-01,
       9.90467216e-01, 9.79551805e-01, 9.56442290e-01, 9.08490135e-01,
       8.13192301e-01, 6.40481916e-01, 3.82931704e-01, 1.26430470e-01,
       1.16141622e-02, 6.77848140e-05, 1.04336599e-09, 4.42993818e-20])

In [23]:
SH4['flux'][0,0,::4,0]

array([1.00000000e+00, 9.99973171e-01, 9.99915372e-01, 9.99790865e-01,
       9.99522693e-01, 9.98945270e-01, 9.97702794e-01, 9.95033122e-01,
       9.89314515e-01, 9.77145449e-01, 9.51610728e-01, 8.99582682e-01,
       7.99718376e-01, 6.28698907e-01, 3.88424595e-01, 1.46195555e-01,
       1.84723381e-02, 2.15220478e-04, 1.46892948e-08, 1.53166644e-17])

In [28]:
Cdisort[b'flux_down'][0,0,:,0]

array([1.00000000e+00, 9.99967799e-01, 9.99898444e-01, 9.99749123e-01,
       9.99427877e-01, 9.98737876e-01, 9.97260858e-01, 9.94121040e-01,
       9.87534860e-01, 9.74024734e-01, 9.47081402e-01, 8.94599778e-01,
       7.96042859e-01, 6.28343071e-01, 3.89858048e-01, 1.45762742e-01,
       1.83387413e-02, 2.16660619e-04, 1.52947008e-08, 1.71170754e-17])

In [ ]:
def L2_error(a,b):
    error = np.sqrt(np.sum((a-b)**2))/np.sqrt(np.sum(b**2))
    return error

In [30]:
from bokeh.palettes import magma
import pickle as pk

def plot_intensity(models, u0=None):
    n=0
#     if u0==None:
#         tit = 'same ubars, ' + models[0]
#     else:
#         tit = 'u0 = ' + str(u0) + ': ' + models[0]
#     if len(models)>1:
#         for j in range(len(models)-1):
#             tit = tit + '+' + models[j+1]
    fig = figure(y_axis_type='log', y_range=[1e-4, 2e-1],
                    x_axis_label = 'w0', y_axis_label = 'intensity',
                    width=500, height=400)#, title=tit)
    w0s = [0.5,0.55,0.6,0.65,0.7,0.8,0.9,0.95,0.98,0.99,0.995,0.999,1.]
    g0s = [0.5,0.75,0.8,0.85,0.9]
    col = magma(len(g0s)+1)
    for g0, i in zip(g0s, range(len(g0s))):
        y1 = []; y2 = []; y3 = []
        for w0 in w0s:
            dir = '/Users/crooney/Documents/codes/pyDISORT/test/'
            if u0==None:
                filename = '/data_%.3f_%.3f.pk' % (g0, w0)
            else:
                filename = '/test/no_direct/data_test_%.3f_%.3f_%.1f.pk' % (g0, w0, u0)


            
            if any('cdisort' in s for s in models):
                output = pk.load(open(dir + 'cdisort_data' + filename,'rb'), encoding = 'bytes')
                from IPython import embed; embed()
                y1.append(output[b'xint_at_top'][n,0,0])
            if any('picaso' in s for s in models):
                output = pk.load(open(dir + 'picaso_data' + filename,'rb'), encoding = 'bytes')
                y2.append(output['xint_at_top'][n,0,0])
            if any('SH4' in s for s in models):
                output = pk.load(open(dir + 'picaso_data/SH4' + filename,'rb'), encoding = 'bytes')
                y3.append(output['xint_at_top'][n,0,0])
                
        if y1:
            fig.line(w0s, y1, line_width=3, color=col[i], legend_label='g0 = %.1f' % g0)
        if y2:
            fig.line(w0s, y2, line_width=3, color=col[i], line_dash='dotted',legend_label='g0 = %.1f' % g0)
        if y3:
            fig.line(w0s, y3, line_width=3, color=col[i], line_dash='dashed',legend_label='g0 = %.1f' % g0)

    jpi.plot_format(fig)
    fig.legend.location = 'top_left'
    fig.add_layout(fig.legend[0], 'right')
    return fig
# show(fig)

In [ ]:
cdisort_dir = '/Users/crooney/Documents/codes/picaso/docs/notebooks/cdisort_data/test/'
fig = plot_intensity( ['cdisort','SH4', 'picaso'])
fig2 = plot_intensity( ['cdisort','SH4', 'picaso'])

Python 3.8.5 (default, Sep  4 2020, 02:22:02) 
Type 'copyright', 'credits' or 'license' for more information
IPython 7.18.1 -- An enhanced Interactive Python. Type '?' for help.

In [1]: output.keys()
Out[1]: dict_keys([b'nwno', b'cos_theta', b'tweight', b'gweight', b'tau', b'flux_up', b'xint_at_top', b'flux_down', b'F0PI'])



In [21]:
from bokeh.layouts import row
show(row(fig,fig2))

In [17]:
from bokeh.palettes import magma
import pickle as pk

def plot_flux(models, w0, g0s, net=False):
    n=0
    col = magma(len(g0s)+1)
    tit = 'w0 = %.1f' % w0
    fig = figure(y_axis_type='log', y_axis_label = 'opd', y_range=[1e2, 1e0],
                    x_axis_type='log', x_axis_label = 'downward flux',# x_range=[1e-10,inf],
                    width=500, height=500, title=tit)
    fig2 = figure(y_axis_type='log', y_axis_label = 'opd', y_range=[1e2, 1e0],
                    x_axis_type='log', x_axis_label = 'upward flux',
                    width=500, height=500, title=tit)
    fig3 = figure(y_axis_type='log', y_axis_label = 'opd', y_range=[1e2, 1e0],
                    x_axis_type='log', x_axis_label = 'net flux',
                    width=500, height=500, title=tit)
    
    for g0, i in zip(g0s, range(len(g0s))):
        cdisort_up = []; SH4_up = []; picaso_up = []
        cdisort_dwn = []; SH4_dwn= []; picaso_dwn = []
        
        dir = '/Users/crooney/Documents/codes/pyDISORT/test/'
        filename = '/data_%.3f_%.3f.pk' % (g0, w0)


        if any('cdisort' in s for s in models):
            output = pk.load(open(dir + 'cdisort_data' + filename,'rb'), encoding = 'bytes')
            cdisort_up.append(output[b'flux_up'][n,0,:,0])
            cdisort_dwn.append(output[b'flux_down'][n,0,:,0])
            tau = output[b'tau'][:,0]
            fig.line(cdisort_dwn[0], tau, line_width=3, color=col[i], line_dash='solid', legend_label='cdisort')
            fig2.line(cdisort_up[0], tau, line_width=3, color=col[i], line_dash='solid', legend_label='g0 = %.1f' % g0)
            fig3.line(cdisort_dwn[0]-cdisort_up[0], tau, line_width=3, color=col[i], line_dash='solid', legend_label='cdisort, g0 = %.1f' % g0)
         
        if any('picaso' in s for s in models):
            output = pk.load(open(dir + 'picaso_data' + filename,'rb'), encoding = 'bytes')
            picaso_up.append(output['flux'][n,0,1::2,0])
            picaso_dwn.append(output['flux'][n,0,::2,0])
            tau = output['tau'][:,0]
            fig.line(picaso_dwn[0], tau, line_width=3, color=col[i], line_dash='dotted', legend_label='picaso')
            fig2.line(picaso_up[0], tau, line_width=3, color=col[i], line_dash='dotted', legend_label='g0 = %.1f' % g0)
            fig3.line(picaso_dwn[0]-picaso_up[0], tau, line_width=3, color=col[i], line_dash='dotted', legend_label='picaso, g0 = %.1f' % g0)


        if any('SH4' in s for s in models):
            output = pk.load(open(dir + 'picaso_data/SH4' + filename,'rb'), encoding = 'bytes')
            SH4_up.append(output['flux'][n,0,2::4,0])
            SH4_dwn.append(output['flux'][n,0,::4,0])
            tau = output['tau'][:,0]
            fig.line(SH4_dwn[0], tau, line_width=3, color=col[i], line_dash='dashed', legend_label='SH4')
            fig2.line(SH4_up[0], tau, line_width=3, color=col[i], line_dash='dashed', legend_label='g0 = %.1f' % g0)
            fig3.line(SH4_dwn[0]-SH4_up[0], tau, line_width=3, color=col[i], line_dash='dashed', legend_label='SH4, g0 = %.1f' % g0)


            
    fig.legend.location='top_left'
    fig.x_range.start = 1e-10
    jpi.plot_format(fig) 
    
    fig2.legend.location='top_left'
    fig2.x_range.start = 1e-10
    jpi.plot_format(fig2)
    
    fig3.legend.location='top_left'
    fig3.x_range.start = 1e-10
    jpi.plot_format(fig3)

    from bokeh.layouts import row
    fig4= row(fig,fig2)
    
    if net:
        return fig3
    else:
        return fig4
# show(fig)

In [18]:
g0s = [0.0,0.9]
w0 = 0.5; 
fig = plot_flux(['cdisort,''SH4','picaso'], w0, g0s, net=True)
show(fig)